 #                     IBM APPLIED Data Science Capstone Project

#      Segmenting and Clustering Neighborhoods in Toronto


In [34]:
import numpy as np                        # library to handle data in a vectorized manner

import pandas as pd                       # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json                               # library to handle JSON files

!conda install -c conda-forge geopy --yes           # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim               # convert an address into latitude and longitude values

import requests                                    # library to handle requests
from pandas.io.json import json_normalize                 # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes          # uncomment this line if you haven't completed the Foursquare API lab
import folium                                             # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


#   WebScraping Toronto Boroughs & Pin-Codes.


Install Beautiful Soup Package & Continue....

Extract the WikiTable with 287 rows.

In [35]:
from bs4 import BeautifulSoup
import requests
import csv

source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'lxml')

table=soup.table.tbody

table_df=pd.DataFrame()

for string in table.stripped_strings:
    table_df=table_df.append([string])

Borough_df=pd.DataFrame(table_df.to_numpy().reshape(-1,3))
Borough_df.columns = Borough_df.iloc[0]
Borough_df=Borough_df.drop(0).reset_index(drop=True)
Borough_df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Not assigned


### Now that we have the required table. Let us remove 'Not assigned' [210 rows remain]  and Data Wrangling
After Wrangling, we are left with 103 unique postal codes...!

In [36]:
#Remove 'NA' Borough, if Neighbourhood="NA", use Borough value
Borough_df=Borough_df[Borough_df['Borough']!='Not assigned'].reset_index(drop=True)

Borough_df=Borough_df.replace('Not assigned',np.NaN)
Borough_df["COL3"] = Borough_df["Neighbourhood"].fillna(Borough_df["Borough"])
Borough_df=Borough_df.drop(['Neighbourhood'],axis=1).reset_index(drop=True)
Borough_df. rename (columns={'COL3':'Neighbourhood'}, inplace=True)

#Group by Postcode & Borough with Concatenation of Neighbourhood Vales

Grouped_df=Borough_df.groupby(['Postcode','Borough'],as_index=False).agg(', '.join)
print(Grouped_df.head())

print('\n \n The dataframe has {} boroughs \n and \n {} ROWS .'.format(
        len(Grouped_df['Borough'].unique()),
        Grouped_df.shape[0]))

0 Postcode      Borough                           Neighbourhood
0      M1B  Scarborough                          Rouge, Malvern
1      M1C  Scarborough  Highland Creek, Rouge Hill, Port Union
2      M1E  Scarborough       Guildwood, Morningside, West Hill
3      M1G  Scarborough                                  Woburn
4      M1H  Scarborough                               Cedarbrae

 
 The dataframe has 11 boroughs 
 and 
 103 ROWS .


### Fetch Location data from the given link & Join with boroughs data [Tor_df]  

In [37]:
loc_df = pd.read_csv ('http://cocl.us/Geospatial_data')
loc_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [38]:
Tor_df = pd.merge(Grouped_df, loc_df, left_on='Postcode', right_on='Postal Code', how='left')
Tor_df=Tor_df.drop(['Postal Code'],axis=1).reset_index(drop=True)
Tor_df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


## Create a map of Toronto Neighbourhoods using FOLIUM

Use Geopy to get Location of Toronto Neighbourhoods

Create Neighbourhood Map of Toronto

In [39]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


## Select only Toronto Boroughs

In [40]:
NH_df=Tor_df[Tor_df['Borough'].str.contains('Toronto')].sort_values (by = 'Borough').drop(['Postcode'],axis=1).reset_index(drop=True)
NH_df

,Borough,Neighbourhood,Latitude,Longitude
0,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678
1,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307
2,Central Toronto,Lawrence Park,43.728020,-79.388790
3,Central Toronto,Davisville North,43.712751,-79.390197
4,Central Toronto,North Toronto West,43.715383,-79.405678
5,Central Toronto,Davisville,43.704324,-79.388790
6,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
7,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049
8,Central Toronto,Roselawn,43.711695,-79.416936
9,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752


In [47]:
# create map of Toronto with Boroughs using latitude and longitude values
map_tor = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(NH_df['Latitude'], NH_df['Longitude'], NH_df['Borough'], NH_df['Neighbourhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor

Enter the Fourquare credentials

In [57]:
CLIENT_ID = 'GQYZTHDW4FELZGYMP0DIQXVDFGXO5MMAVZATGRXRFKCPJV4X' # your Foursquare ID
CLIENT_SECRET = 'IWPETGS0XZIYMQBDYOZO2QRWDE0UJFHOBRA1YPTL1PK3F4SL' # your Foursquare Secret
VERSION = '20190202'
LIMIT = 100
print('Your credentails have been entered')


Your credentails have been entered


##### TEST if we can obtain data from Fourquare:

Get Restaurants data from FourSquare Toronto City are 43.653963, -79.387207.

In [93]:
search_query = 'Restaurant'  #['Restaurant','School','Bar','Medical','Shop']
radius = 500


url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

results = requests.get(url).json()


In [94]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
rest_df = json_normalize(venues)
rest_df.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",False,4ad4c05ff964a52048f720e3,110 Chestnut Street,CA,Toronto,Canada,NaN,145,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]","[{'label': 'display', 'lat': 43.65488413420439...",43.654884,-79.385931,M5G 1R3,ON,Hemispheres Restaurant & Bistro,v-1582556329,NaN
1,"[{'id': '52af3b773cf9994f4e043c03', 'name': 'S...",False,5750b013498e755287c6de97,988 Baldwin Street,CA,Toronto,Canada,NaN,573,"[988 Baldwin Street, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.655874, 'lng':...",43.655874,-79.393826,NaN,ON,Some Time BBQ Grill Restaurant 碳烤屋,v-1582556329,NaN
2,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",False,4b2027b5f964a520f82d24e3,195 Dundas St W,CA,Toronto,Canada,at University Ave,107,"[195 Dundas St W (at University Ave), Toronto ...","[{'label': 'display', 'lat': 43.65492521335936...",43.654925,-79.387089,M5G 1C7,ON,Hong Shing Chinese Restaurant,v-1582556329,60327598
3,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",False,50ca02c0245f2d4aa8c2b313,145 Richmond Street West,CA,Toronto,Canada,Hilton Toronto,458,"[145 Richmond Street West (Hilton Toronto), To...","[{'label': 'display', 'lat': 43.65000998764964...",43.650010,-79.385608,M5H 2L2,ON,Tundra Restaurant,v-1582556329,NaN
4,"[{'id': '4bf58dd8d48988d14a941735', 'name': 'V...",False,4c476d6719fde21e32410876,179 Dundas St. W.,CA,Toronto,Canada,at Chestnut,140,"[179 Dundas St. W. (at Chestnut), Toronto ON M...","[{'label': 'display', 'lat': 43.65506808, 'lng...",43.655068,-79.386375,M5G,ON,Cali Restaurant,v-1582556329,NaN


In [95]:
rest_df=rest_df[['name','location.lat','location.lng']]
rest_df['name'].nunique()

40

###### There are 40 restaurants within 500m of Toronto City Center.

###### Get data for Number of Restaurants, Bars, Schools, Medical Centres & Shops for all the Neighbourhoods of TORONTO.

###### For this repetitive process, Create a user-defined function to Get Venues & Store them in Data Frame !

In [96]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

            
        # make the GET request
        results = requests.get(url).json()
        venues = results['response']['venues']

        # tranform venues into a dataframe
        #rest_df = json_normalize(venues)
        #rest_df=rest_df[['name']]
        
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            lat,
            lng,
            v['name']) for v in venues])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue']
    
    return(nearby_venues)

##### Use the user-defined function to get Data for all Neighbourhoods.

In [104]:
Tor_venues = getNearbyVenues(names=NH_df['Neighbourhood'],
                                   latitudes=NH_df['Latitude'],
                                   longitudes=NH_df['Longitude']
                                  )
Tor_venues

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue
0,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,Hemispheres Restaurant & Bistro
1,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,Some Time BBQ Grill Restaurant 碳烤屋
2,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,Hong Shing Chinese Restaurant
3,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,Tundra Restaurant
4,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,Yueh Tung Chinese Restaurant
5,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,Wah Too Seafood Restaurant
6,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,Cali Restaurant
7,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,New Treasure Restaurant
8,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,Anoush Restaurant
9,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,Adega Restaurant


In [106]:
Tor_Group=Tor_venues.groupby(['Neighbourhood'],as_index=False).count()

Tor_Group

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue
0,"Adelaide, King, Richmond",40,40,40
1,Berczy Park,40,40,40
2,"Brockton, Exhibition Place, Parkdale Village",40,40,40
3,Business Reply Mail Processing Centre 969 Eastern,40,40,40
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",40,40,40
5,"Cabbagetown, St. James Town",40,40,40
6,Central Bay Street,40,40,40
7,"Chinatown, Grange Park, Kensington Market",40,40,40
8,Christie,40,40,40
9,Church and Wellesley,40,40,40
